In [1]:
import torch
from torch import optim
from torch.distributions import MultivariateNormal
import argparse
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torchvision import datasets
import torchvision.transforms as transforms
from dataset import DATASETS
from benchmarks import *
import matplotlib.pyplot as plt
from vae import *
from plotting import *
import tqdm
from datetime import datetime
import pandas as pd
from training import get_marginal_posterior, test_sklearn, test_vae, train_sklearn, train_vae, validate_sklearn, validate_vae

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [3]:
def configure():
    parser = argparse.ArgumentParser(description='')
    # General flags
    parser.add_argument('--dataset', type=str, default="beth", choices=list(DATASETS.keys()), metavar='D', help='Dataset selection')
    parser.add_argument('--disable-cuda', action='store_true', help='Disable CUDA')
    parser.add_argument('--seed', type=int, default=1, metavar='S', help='Random seed')
    parser.add_argument('--subsample', type=int, default=0, metavar='S', help='Factor by which to subsample the dataset')
    parser.add_argument('--vis-latents', action='store_true', help='True if want to visualise latent space')
    parser.add_argument('--vis', action='store_true', help='True if want to visualise dataset (and each epoch)')
    # Training/Testing flags
    parser.add_argument('--test', action='store_true', help='Test benchmarks')
    parser.add_argument('--train', action='store_true', help='Train benchmarks')
    parser.add_argument('--batch-size', type=int, default=128, metavar='B', help='Minibatch size')
    parser.add_argument('--epochs', type=int, default=20, metavar='N', help='Training epochs')
    parser.add_argument('--patience', type=int, default=3, metavar='P', help='Early stopping patience')
    # Model flags
    parser.add_argument('--benchmark', type=str, default="rcov", choices=BENCHMARK_LIST, help='Override fitting of VAE model with specified benchmark')
    parser.add_argument('--outliers-fraction', type=float, default=0.05, help='Assumed proportion of the data that is an outlier') # used in rcov and ifor
    # # VAE
    parser.add_argument('--latent-size', type=int, default=2, metavar='Z', help='Latent size')
    parser.add_argument('--hidden-size', type=int, default=64, metavar='H', help='Hidden size')
    parser.add_argument('--learning-rate', type=float, default=0.003, metavar='L', help='Learning rate')
    parser.add_argument('--weight-decay', type=float, default=0.1, metavar='W', help='Weight decay')
    args = parser.parse_args(['--train', '--benchmark', 'dose'])
    use_cuda = torch.cuda.is_available() and not args.disable_cuda
    args.device = torch.device('cuda' if use_cuda else 'cpu')
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    if use_cuda:
        torch.cuda.manual_seed(args.seed)
    return args


In [4]:
args = configure()
train_dataset, val_dataset, test_dataset = [DATASETS[args.dataset](split=split, subsample=args.subsample) for split in ["train", "val", "test"]]
input_shape = train_dataset.get_input_shape()


In [5]:
model = VAE(input_shape=input_shape, latent_size=2, hidden_size=64, observation=train_dataset.get_distribution())
model.to(device)
optimiser = optim.AdamW(model.parameters(), lr=args.learning_rate, weight_decay=args.weight_decay)
prior = MultivariateNormal(torch.zeros(args.latent_size, device=args.device), torch.eye(args.latent_size, device=args.device))

In [6]:
pth_filename = "results/beth_dose_1.pth"  
with open(pth_filename, 'wb') as file:  
    torch.save(model, file)

In [7]:
mynet = torch.load("results/beth_dose_1.pth")
mynet.eval()

VAE(
  (encoder): EmbeddingEncoder(
    (embeddings): ModuleList(
      (0): Embedding(2, 9)
      (1): Embedding(2, 9)
      (2): Embedding(2, 9)
      (3): Embedding(2, 9)
      (4): Embedding(1011, 9)
      (5): Embedding(6, 9)
      (6): Embedding(3, 9)
    )
    (fc2): Linear(in_features=63, out_features=4, bias=True)
  )
  (decoder): CategoricalDecoder(
    (fc1): Linear(in_features=2, out_features=64, bias=True)
    (fc2): Linear(in_features=64, out_features=1028, bias=True)
  )
)

In [8]:
torch.save(model.state_dict(), "results/beth_dose_1.pth" )
model.load_state_dict(torch.load("results/beth_dose_1.pth" , map_location=lambda storage, loc: storage))

<All keys matched successfully>

In [9]:
model.eval()

VAE(
  (encoder): EmbeddingEncoder(
    (embeddings): ModuleList(
      (0): Embedding(2, 9)
      (1): Embedding(2, 9)
      (2): Embedding(2, 9)
      (3): Embedding(2, 9)
      (4): Embedding(1011, 9)
      (5): Embedding(6, 9)
      (6): Embedding(3, 9)
    )
    (fc2): Linear(in_features=63, out_features=4, bias=True)
  )
  (decoder): CategoricalDecoder(
    (fc1): Linear(in_features=2, out_features=64, bias=True)
    (fc2): Linear(in_features=64, out_features=1028, bias=True)
  )
)

In [15]:
data = pd.read_csv("data/sample_single_safe.csv")
labels = pd.DataFrame(data[["sus"]])
data = pd.DataFrame(data[["processId", "parentProcessId", "userId", "mountNamespace", "eventId", "argsNum", "returnValue"]])
data["processId"] = data["processId"].map(lambda x: 0 if x in [0, 1, 2] else 1)  # Map to OS/not OS
data["parentProcessId"] = data["parentProcessId"].map(lambda x: 0 if x in [0, 1, 2] else 1)  # Map to OS/not OS
data["userId"] = data["userId"].map(lambda x: 0 if x < 1000 else 1)  # Map to OS/not OS
data["mountNamespace"] = data["mountNamespace"].map(lambda x: 0 if x == 4026531840 else 1)  # Map to mount access to mnt/ (all non-OS users) /elsewhere
data["eventId"] = data["eventId"]  # Keep eventId values (requires knowing max value)
data["returnValue"] = data["returnValue"].map(lambda x: 0 if x == 0 else (1 if x > 0 else 2))  # Map to success/success with value/error
# Extract values
data = torch.as_tensor(data.values, dtype=torch.int64)
labels = torch.as_tensor(labels.values, dtype=torch.int64)
total = (data, labels)
print(total)

(tensor([[ 1,  0,  0,  1, 41,  3,  1]]), tensor([[0]]))


In [17]:
decode, predict, z = model(total)

AttributeError: 'tuple' object has no attribute 'cuda'

In [169]:
print(predict)
print(z)

Independent(Normal(loc: torch.Size([1, 2]), scale: torch.Size([1, 2])), 1)
tensor([[-0.1215,  1.0099]], device='cuda:0', grad_fn=<AddBackward0>)


In [191]:
predicted_class = np.argmax(predict)
print("Evil= 1, Safe= 0: ", predicted_class)

Evil= 1, Safe= 0:  0


In [129]:
data = pd.read_csv("data/sample_single_evil.csv")
print(data)
labels = pd.DataFrame(data[["sus"]])
data = pd.DataFrame(data[["processId", "parentProcessId", "userId", "mountNamespace", "eventId", "argsNum", "returnValue"]])
data["processId"] = data["processId"].map(lambda x: 0 if x in [0, 1, 2] else 1)  # Map to OS/not OS
data["parentProcessId"] = data["parentProcessId"].map(lambda x: 0 if x in [0, 1, 2] else 1)  # Map to OS/not OS
data["userId"] = data["userId"].map(lambda x: 0 if x < 1000 else 1)  # Map to OS/not OS
data["mountNamespace"] = data["mountNamespace"].map(lambda x: 0 if x == 4026531840 else 1)  # Map to mount access to mnt/ (all non-OS users) /elsewhere
data["eventId"] = data["eventId"]  # Keep eventId values (requires knowing max value)
data["returnValue"] = data["returnValue"].map(lambda x: 0 if x == 0 else (1 if x > 0 else 2))  # Map to success/success with value/error
# Extract values
data = torch.as_tensor(data.values, dtype=torch.int64)
labels = torch.as_tensor(labels.values, dtype=torch.int64)

print(data)

    timestamp  processId  threadId  parentProcessId  userId  mountNamespace  \
0  496.624883       7555      7555             7548    1001      4026531840   

  processName         hostName  eventId eventName stackAddresses  argsNum  \
0         tsm  ip-10-100-1-217       42   connect             []        3   

   returnValue                                               args  sus  evil  
0         -114  [{'name': 'sockfd', 'type': 'int', 'value': 34...    1     1  
tensor([[ 1,  1,  1,  0, 42,  3,  2]])


In [150]:
decode, predict, z = model(data.cuda())
print(predict.sample())
print(z)

tensor([[-0.1262,  2.2697]], device='cuda:0')
tensor([[-0.2358, -0.0955]], device='cuda:0', grad_fn=<AddBackward0>)


In [188]:
predicted_class = labels[np.argmax(predict)].numpy()[0]
print("Evil= 1, Safe= 0: ", predicted_class)
print(predict)

Evil= 1, Safe= 0:  1
Independent(Normal(loc: torch.Size([1, 2]), scale: torch.Size([1, 2])), 1)
